In [1]:
# train_random_forest_optuna.py – cleaned version (no date strings in X)
"""Daily‑level RandomForest + Optuna (multi‑output)

* Raw `lifelog_date` is **dropped** before the ColumnTransformer.
* Only numeric sensor features + `days_since_start` + one‑hot `subject_id` pass to model.
"""
from __future__ import annotations

import joblib, optuna, pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

AGG_STRATEGY = "mixed"  # sum | mean | mixed | both
TEST_SIZE = 0.30
N_TRIALS = 100
RANDOM_STATE = 42

def load_data():
    merged = pd.read_csv("/Users/jhlee/Desktop/고려대학교/4-1/데이터과학/프로젝트/ETRI_lifelog_dataset/merge_df.csv", parse_dates=["timestamp"])
    train_y = pd.read_csv("/Users/jhlee/Desktop/고려대학교/4-1/데이터과학/프로젝트/ETRI_lifelog_dataset/ch2025_metrics_train.csv", parse_dates=["lifelog_date"])
    submit  = pd.read_csv("/Users/jhlee/Desktop/고려대학교/4-1/데이터과학/프로젝트/ETRI_lifelog_dataset/ch2025_submission_sample.csv", parse_dates=["lifelog_date"])
    return merged, train_y, submit

def aggregate_features(df_raw: pd.DataFrame, strategy="mixed") -> pd.DataFrame:
    df = df_raw.copy()
    df["lifelog_date"] = df["timestamp"].dt.floor("D")
    id_cols = ["subject_id", "lifelog_date"]
    feat_cols = [c for c in df.columns if c not in id_cols + ["timestamp"]]
    if strategy == "sum":
        agg_rules = {c: "sum" for c in feat_cols}
    elif strategy == "mean":
        agg_rules = {c: "mean" for c in feat_cols}
    elif strategy == "mixed":
        sum_like = ["time", "distance", "calorie", "count", "usage"]
        agg_rules = {c: ("sum" if any(k in c.lower() for k in sum_like) else "mean") for c in feat_cols}
    elif strategy == "both":
        sum_df = df.groupby(id_cols)[feat_cols].sum().add_suffix("_sum")
        mean_df = df.groupby(id_cols)[feat_cols].mean().add_suffix("_mean")
        return sum_df.join(mean_df).reset_index()
    else:
        raise ValueError("Invalid AGG_STRATEGY")
    return df.groupby(id_cols).agg(agg_rules).reset_index()

def build_preprocessor(cat_cols, num_cols):
    return ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ], remainder="drop")

def build_model(trial, pre):
    rf = RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),
        max_depth=trial.suggest_int("max_depth", 6, 30),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 8),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        bootstrap=trial.suggest_categorical("bootstrap", [True, False]),
        random_state=RANDOM_STATE, n_jobs=-1)
    return Pipeline([("pre", pre), ("clf", MultiOutputClassifier(rf))])

def macro_metrics(y_true, y_pred):
    acc = accuracy_score(y_true.values.ravel(), y_pred.ravel())
    f1 = f1_score(y_true.values.ravel(), y_pred.ravel(), average="macro")
    return acc, f1

def objective(trial, X_tr, X_val, y_tr, y_val, pre):
    model = build_model(trial, pre)
    model.fit(X_tr, y_tr)
    preds = model.predict(X_val)
    acc, f1 = macro_metrics(y_val, preds)
    trial.set_user_attr("accuracy", acc)
    trial.set_user_attr("macro_f1", f1)
    return (acc + f1) / 2

def main():
    merged, train_y, submit = load_data()
    feats = aggregate_features(merged, AGG_STRATEGY)
    data = train_y.merge(feats, on=["subject_id", "lifelog_date"], how="inner")
    data["days_since_start"] = (data["lifelog_date"] - data["lifelog_date"].min()).dt.days
    target_cols = ["Q1", "Q2", "Q3", "S1", "S2", "S3"]

    # ---------------- explicit feature filtering ----------------
    base_features = [c for c in data.columns if c not in target_cols + ["lifelog_date"]]
    cat_cols = ["subject_id"]
    num_cols = [c for c in base_features if c != "subject_id" and pd.api.types.is_numeric_dtype(data[c])]
    feature_cols = cat_cols + num_cols  # guarantee lifelog_date absent

    pre = build_preprocessor(cat_cols, num_cols)
    X = data[feature_cols]
    y = data[target_cols]
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y["Q1"])

    study = optuna.create_study(direction="maximize", study_name="RF_multioutput")
    study.optimize(lambda t: objective(t, X_tr, X_val, y_tr, y_val, pre), n_trials=N_TRIALS, show_progress_bar=True)

    print("Best accuracy:", study.best_trial.user_attrs["accuracy"])
    print("Best macro‑F1:", study.best_trial.user_attrs["macro_f1"])

    best_model = build_model(optuna.trial.FixedTrial(study.best_params), pre)
    best_model.fit(X, y)
    joblib.dump(best_model, "rf_multioutput_best.pkl")

    # -------- submission set
    sub = submit.merge(feats, on=["subject_id", "lifelog_date"], how="left")
    sub["days_since_start"] = (sub["lifelog_date"] - data["lifelog_date"].min()).dt.days
    X_sub = sub[feature_cols]
    submit[target_cols] = best_model.predict(X_sub)
    submit.to_csv("ch2025_submission_rf.csv", index=False)
    print("Saved → ch2025_submission_rf.csv")

if __name__ == "__main__":
    main()


[I 2025-05-20 16:04:55,308] A new study created in memory with name: RF_multioutput


  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3994980883.py:57: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


[I 2025-05-20 16:04:56,233] Trial 0 finished with value: 0.5426800866240364 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.5426800866240364.


/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3994980883.py:57: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


...

[I 2025-05-20 16:07:24,074] Trial 98 finished with value: 0.5442662423847721 and parameters: {'n_estimators': 200, 'max_depth': 21, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 70 with value: 0.5537309346212612.


/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3994980883.py:57: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


[I 2025-05-20 16:07:25,294] Trial 99 finished with value: 0.5532244510735702 and parameters: {'n_estimators': 300, 'max_depth': 24, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 70 with value: 0.5537309346212612.
Best accuracy: 0.616
Best macro‑F1: 0.4914618692425224


/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3994980883.py:57: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


Saved → ch2025_submission_rf.csv


In [2]:
# train_random_forest_optuna.py – cleaned version (no date strings in X)
"""Daily‑level RandomForest + Optuna (multi‑output)

* Raw `lifelog_date` is **dropped** before the ColumnTransformer.
* Only numeric sensor features + `days_since_start` + one‑hot `subject_id` pass to model.
"""
from __future__ import annotations

import joblib, optuna, pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

AGG_STRATEGY = "mixed"  # sum | mean | mixed | both
TEST_SIZE = 0.30
N_TRIALS = 100
RANDOM_STATE = 42

def load_data():
    merged = pd.read_csv("/Users/jhlee/Desktop/고려대학교/4-1/데이터과학/프로젝트/ETRI_lifelog_dataset/merge_df.csv", parse_dates=["timestamp"])
    train_y = pd.read_csv("/Users/jhlee/Desktop/고려대학교/4-1/데이터과학/프로젝트/ETRI_lifelog_dataset/ch2025_metrics_train.csv", parse_dates=["lifelog_date"])
    submit  = pd.read_csv("/Users/jhlee/Desktop/고려대학교/4-1/데이터과학/프로젝트/ETRI_lifelog_dataset/ch2025_submission_sample.csv", parse_dates=["lifelog_date"])
    return merged, train_y, submit

def aggregate_features(df_raw: pd.DataFrame, strategy="mixed") -> pd.DataFrame:
    df = df_raw.copy()
    df["lifelog_date"] = df["timestamp"].dt.floor("D")
    id_cols = ["subject_id", "lifelog_date"]
    feat_cols = [c for c in df.columns if c not in id_cols + ["timestamp"]]
    if strategy == "sum":
        agg_rules = {c: "sum" for c in feat_cols}
    elif strategy == "mean":
        agg_rules = {c: "mean" for c in feat_cols}
    elif strategy == "mixed":
        sum_like = ["time", "distance", "calorie", "count", "usage"]
        agg_rules = {c: ("sum" if any(k in c.lower() for k in sum_like) else "mean") for c in feat_cols}
    elif strategy == "both":
        sum_df = df.groupby(id_cols)[feat_cols].sum().add_suffix("_sum")
        mean_df = df.groupby(id_cols)[feat_cols].mean().add_suffix("_mean")
        return sum_df.join(mean_df).reset_index()
    else:
        raise ValueError("Invalid AGG_STRATEGY")
    return df.groupby(id_cols).agg(agg_rules).reset_index()

def build_preprocessor(cat_cols, num_cols):
    return ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ], remainder="drop")

def build_model(trial, pre):
    rf = RandomForestClassifier(
        criterion = "entropy",
        n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),
        max_depth=trial.suggest_int("max_depth", 6, 30),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 8),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        bootstrap=trial.suggest_categorical("bootstrap", [True, False]),
        random_state=RANDOM_STATE, n_jobs=-1)
    return Pipeline([("pre", pre), ("clf", MultiOutputClassifier(rf))])

def macro_metrics(y_true, y_pred):
    acc = accuracy_score(y_true.values.ravel(), y_pred.ravel())
    f1 = f1_score(y_true.values.ravel(), y_pred.ravel(), average="macro")
    return acc, f1

def objective(trial, X_tr, X_val, y_tr, y_val, pre):
    model = build_model(trial, pre)
    model.fit(X_tr, y_tr)
    preds = model.predict(X_val)
    acc, f1 = macro_metrics(y_val, preds)
    trial.set_user_attr("accuracy", acc)
    trial.set_user_attr("macro_f1", f1)
    return (acc + f1) / 2

def main():
    merged, train_y, submit = load_data()
    feats = aggregate_features(merged, AGG_STRATEGY)
    data = train_y.merge(feats, on=["subject_id", "lifelog_date"], how="inner")
    data["days_since_start"] = (data["lifelog_date"] - data["lifelog_date"].min()).dt.days
    target_cols = ["Q1", "Q2", "Q3", "S1", "S2", "S3"]

    # ---------------- explicit feature filtering ----------------
    base_features = [c for c in data.columns if c not in target_cols + ["lifelog_date"]]
    cat_cols = ["subject_id"]
    num_cols = [c for c in base_features if c != "subject_id" and pd.api.types.is_numeric_dtype(data[c])]
    feature_cols = cat_cols + num_cols  # guarantee lifelog_date absent

    pre = build_preprocessor(cat_cols, num_cols)
    X = data[feature_cols]
    y = data[target_cols]
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y["Q1"])

    study = optuna.create_study(direction="maximize", study_name="RF_multioutput")
    study.optimize(lambda t: objective(t, X_tr, X_val, y_tr, y_val, pre), n_trials=N_TRIALS, show_progress_bar=True)

    print("Best accuracy:", study.best_trial.user_attrs["accuracy"])
    print("Best macro‑F1:", study.best_trial.user_attrs["macro_f1"])

    best_model = build_model(optuna.trial.FixedTrial(study.best_params), pre)
    best_model.fit(X, y)
    joblib.dump(best_model, "rf_multioutput_best.pkl")

    # -------- submission set
    sub = submit.merge(feats, on=["subject_id", "lifelog_date"], how="left")
    sub["days_since_start"] = (sub["lifelog_date"] - data["lifelog_date"].min()).dt.days
    X_sub = sub[feature_cols]
    submit[target_cols] = best_model.predict(X_sub)
    submit.to_csv("ch2025_submission_rf_entropy.csv", index=False)
    print("Saved → ch2025_submission_rf_entropy.csv")

if __name__ == "__main__":
    main()


[I 2025-05-20 16:07:27,058] A new study created in memory with name: RF_multioutput


  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3698124840.py:58: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


[I 2025-05-20 16:07:29,839] Trial 0 finished with value: 0.5275231409894543 and parameters: {'n_estimators': 500, 'max_depth': 11, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 0.5275231409894543.


/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3698124840.py:58: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


...

[I 2025-05-20 16:10:29,239] Trial 98 finished with value: 0.5412409307124105 and parameters: {'n_estimators': 200, 'max_depth': 7, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 97 with value: 0.559435756463983.


/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3698124840.py:58: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


[I 2025-05-20 16:10:30,091] Trial 99 finished with value: 0.559435756463983 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 97 with value: 0.559435756463983.
Best accuracy: 0.6266666666666667
Best macro‑F1: 0.4922048462612993


/var/folders/hq/_frv_ntj02d_ynk43wvy6n780000gn/T/ipykernel_71848/3698124840.py:58: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators=trial.suggest_int("n_estimators", 200, 800, 100),


Saved → ch2025_submission_rf_entropy.csv
